In [10]:
!pip install category_encoders

   ---------------------------------------- 0.0/85.7 kB ? eta -:--:--
   ---------------------------------------- 85.7/85.7 kB 4.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from  sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, mean_absolute_percentage_error, mean_squared_error, r2_score,mean_absolute_error
!pip install optuna-integration[catboost]
#from  typing import
from scipy import sparse
import warnings
warnings.filterwarnings('ignore')
import optuna
import logging
import sys
from optuna.integration.xgboost import XGBoostPruningCallback
# EarlyStopping is directly from XGBoost's own callback module
from xgboost.callback import EarlyStopping

In [4]:
df=pd.read_csv("data/cleaned_data_after_imputation.csv")
df.columns
df.dtypes
numeric_columns=["bedroomCount","toilet_and_bath","habitableSurface","facedeCount","hasTerrace","totalParkingCount"]
categorical_columns=["type","subtype","province","locality","postCode","buildingCondition","epcScore"]

encoder = OneHotEncoder(sparse_output=False, drop="first")
one_hot_encoded = encoder.fit_transform(df[categorical_columns])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))
df = pd.concat([df, one_hot_df], axis=1)
df = df.drop(categorical_columns, axis=1)

In [ ]:
#CATBOOST OPTUNA WITH ONEHOTENCODING
price_column = df['price'] # Store price before dropping categorical columns

# Now, drop the original categorical columns and set X and y
# (The previous concat operation handled this by only taking numeric and one-hot)
# So, X should be the combined numeric and one-hot df
X = df.copy()
# Re-add price column from original df if it was removed
# Assuming price was NOT in `numeric_columns` or `categorical_columns`
# and it was implicitly left out of the `df_numeric` selection.
# If `price` is truly only the target, it shouldn't be in `X`.
if 'price' in X.columns:
    y = X.pop("price") # Extract price as target and remove from features
else:
    # This scenario means 'price' was likely excluded from numeric_columns
    # and not part of categorical_columns. We need to get it from the original df.
    y = pd.read_csv("data/nonull_smalldata.csv")["price"]
    # Adjust X to ensure index alignment with y if `df` (X) was re-created without `price`
    X = df.copy() # X now contains only features

# Align indices just in case
X.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)


print("\nFeatures (X) shape after encoding:", X.shape)
print("Target (y) shape:", y.shape)

# Split data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=1234)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1234)

print("\nData splitting complete.")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# CatBoost's strength is handling categorical features internally.
# However, since you've already One-Hot Encoded them, CatBoost will treat them
# as numerical features. This is perfectly fine, but you won't use CatBoost's
# native categorical handling for these columns.
# If you wanted to use native handling, you would pass `df[categorical_columns]` directly
# and specify `cat_features` indices *before* one-hot encoding.
# For this setup, there are no `cat_features` to pass to Pool, as they are already encoded.
categorical_features_indices = []

# Create CatBoost Pool objects for efficient data handling
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
val_pool = Pool(X_val, y_val, cat_features=categorical_features_indices)
test_pool = Pool(X_test, y_test, cat_features=categorical_features_indices)

print("--- Data Preprocessing Complete ---")

# --- 2. Define the Optuna Objective Function ---
def objective(trial):
    """
    Objective function for Optuna to optimize CatBoost hyperparameters.
    It returns the validation RMSE for a given set of hyperparameters.
    """
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 0.3),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-2, 10),
        "subsample": trial.suggest_uniform("subsample", 0.6, 1.0),
        "colsample_bylevel": trial.suggest_uniform("colsample_bylevel", 0.6, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 30),
        "loss_function": "RMSE",
        "eval_metric": "RMSE",
        "random_seed": 42,
        "verbose": 0,
        #"od_type": "Iter",
        #"od_wait": trial.suggest_int("od_wait", 10, 50),
        "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 10, 50),
    }

    model = CatBoostRegressor(**params)

    pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")

    try:
        model.fit(
            train_pool,
            eval_set=val_pool,
            early_stopping_rounds=params["early_stopping_rounds"],
            callbacks=[pruning_callback],
            verbose=0
        )
    except optuna.exceptions.TrialPruned:
        raise

    val_predictions = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, val_predictions))

    return rmse

# --- 3. Set up and Run Optuna Study ---
if __name__ == "__main__":
    optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(seed=42),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
    )

    print("\n--- Starting Optuna optimization ---")
    study.optimize(objective, n_trials=50, timeout=600) # Run 50 trials or for 10 minutes

    print("\n--- Optimization finished ---")
    print(f"Number of finished trials: {len(study.trials)}")
    print(f"Number of pruned trials: {len(study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.PRUNED]))}")

    print("\nBest trial:")
    trial = study.best_trial

    print(f"  Value (RMSE on validation set): {trial.value:.4f}")
    print("  Best Hyperparameters:")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    # --- 4. Train the final model with the best hyperparameters ---
    print("\n--- Training final model with best hyperparameters ---")
    best_params = trial.params
    best_params_for_final_model = {k: v for k, v in best_params.items() if k not in ["od_wait", "early_stopping_rounds"]}
    best_params_for_final_model["iterations"] = 10000 # Set a high number, rely on early stopping
    best_params_for_final_model["verbose"] = 100 # Show some progress
    best_params_for_final_model["early_stopping_rounds"] = 50 # Re-add for final training

    final_cat_regressor = CatBoostRegressor(**best_params_for_final_model)

    final_cat_regressor.fit(
        train_pool,
        eval_set=val_pool,
        early_stopping_rounds=best_params_for_final_model["early_stopping_rounds"],
        verbose=best_params_for_final_model["verbose"],
    )

    print(f"Best iteration of final model: {final_cat_regressor.get_best_iteration()}")

    # --- 5. Evaluate the final model on the unseen test set ---
    final_predictions = final_cat_regressor.predict(test_pool)

    final_mse = mean_squared_error(y_test, final_predictions)
    final_rmse = np.sqrt(final_mse)
    final_r2 = r2_score(y_test, final_predictions)

    print(f"\n--- Final Model Performance on Test Set ---")
    print(f"  MSE: {final_mse:.4f}")
    print(f"  RMSE: {final_rmse:.4f}")
    print(f"  R-squared: {final_r2:.4f}")

    # Optionally, visualize the optimization process
    try:
        from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice
        print("\n--- Generating Optuna Visualizations (requires plotly and kaleido) ---")
        fig_history = plot_optimization_history(study)
        fig_history.show()

        fig_parallel = plot_parallel_coordinate(study)
        fig_parallel.show()

        fig_slice = plot_slice(study)
        fig_slice.show()
    except ImportError:
        print("\nInstall plotly and kaleido for Optuna visualizations: pip install plotly kaleido")


Features (X) shape after encoding: (56379, 4620)
Target (y) shape: (56379,)

Data splitting complete.
X_train shape: (39465, 4620), y_train shape: (39465,)
X_val shape: (8457, 4620), y_val shape: (8457,)
X_test shape: (8457, 4620), y_test shape: (8457,)


[I 2025-06-30 17:20:24,186] A new study created in memory with name: no-name-bd89960a-8217-4837-86e4-23de06852684


--- Data Preprocessing Complete ---
A new study created in memory with name: no-name-bd89960a-8217-4837-86e4-23de06852684
A new study created in memory with name: no-name-bd89960a-8217-4837-86e4-23de06852684
A new study created in memory with name: no-name-bd89960a-8217-4837-86e4-23de06852684
A new study created in memory with name: no-name-bd89960a-8217-4837-86e4-23de06852684

--- Starting Optuna optimization ---


[I 2025-06-30 17:20:51,264] Trial 0 finished with value: 193164.08466508074 and parameters: {'iterations': 437, 'learning_rate': 0.22648248189516848, 'depth': 9, 'l2_leaf_reg': 0.6251373574521749, 'subsample': 0.6624074561769746, 'colsample_bylevel': 0.662397808134481, 'min_data_in_leaf': 2, 'early_stopping_rounds': 45}. Best is trial 0 with value: 193164.08466508074.


Trial 0 finished with value: 193164.08466508074 and parameters: {'iterations': 437, 'learning_rate': 0.22648248189516848, 'depth': 9, 'l2_leaf_reg': 0.6251373574521749, 'subsample': 0.6624074561769746, 'colsample_bylevel': 0.662397808134481, 'min_data_in_leaf': 2, 'early_stopping_rounds': 45}. Best is trial 0 with value: 193164.08466508074.
Trial 0 finished with value: 193164.08466508074 and parameters: {'iterations': 437, 'learning_rate': 0.22648248189516848, 'depth': 9, 'l2_leaf_reg': 0.6251373574521749, 'subsample': 0.6624074561769746, 'colsample_bylevel': 0.662397808134481, 'min_data_in_leaf': 2, 'early_stopping_rounds': 45}. Best is trial 0 with value: 193164.08466508074.
Trial 0 finished with value: 193164.08466508074 and parameters: {'iterations': 437, 'learning_rate': 0.22648248189516848, 'depth': 9, 'l2_leaf_reg': 0.6251373574521749, 'subsample': 0.6624074561769746, 'colsample_bylevel': 0.662397808134481, 'min_data_in_leaf': 2, 'early_stopping_rounds': 45}. Best is trial 0 wit

...

Trial 47 finished with value: 424164.5951591037 and parameters: {'iterations': 950, 'learning_rate': 0.0011455971996089888, 'depth': 9, 'l2_leaf_reg': 1.9792772454519436, 'subsample': 0.6400316993833451, 'colsample_bylevel': 0.7578869068699705, 'min_data_in_leaf': 29, 'early_stopping_rounds': 38}. Best is trial 43 with value: 191104.37863729885.
Trial 47 finished with value: 424164.5951591037 and parameters: {'iterations': 950, 'learning_rate': 0.0011455971996089888, 'depth': 9, 'l2_leaf_reg': 1.9792772454519436, 'subsample': 0.6400316993833451, 'colsample_bylevel': 0.7578869068699705, 'min_data_in_leaf': 29, 'early_stopping_rounds': 38}. Best is trial 43 with value: 191104.37863729885.
Trial 47 finished with value: 424164.5951591037 and parameters: {'iterations': 950, 'learning_rate': 0.0011455971996089888, 'depth': 9, 'l2_leaf_reg': 1.9792772454519436, 'subsample': 0.6400316993833451, 'colsample_bylevel': 0.7578869068699705, 'min_data_in_leaf': 29, 'early_stopping_rounds': 38}. Best 

[I 2025-06-30 17:29:12,954] Trial 48 finished with value: 202144.32793141753 and parameters: {'iterations': 780, 'learning_rate': 0.24056800772739648, 'depth': 10, 'l2_leaf_reg': 2.9151208029052, 'subsample': 0.6003847334191734, 'colsample_bylevel': 0.7395838685537687, 'min_data_in_leaf': 27, 'early_stopping_rounds': 28}. Best is trial 43 with value: 191104.37863729885.


Trial 48 finished with value: 202144.32793141753 and parameters: {'iterations': 780, 'learning_rate': 0.24056800772739648, 'depth': 10, 'l2_leaf_reg': 2.9151208029052, 'subsample': 0.6003847334191734, 'colsample_bylevel': 0.7395838685537687, 'min_data_in_leaf': 27, 'early_stopping_rounds': 28}. Best is trial 43 with value: 191104.37863729885.
Trial 48 finished with value: 202144.32793141753 and parameters: {'iterations': 780, 'learning_rate': 0.24056800772739648, 'depth': 10, 'l2_leaf_reg': 2.9151208029052, 'subsample': 0.6003847334191734, 'colsample_bylevel': 0.7395838685537687, 'min_data_in_leaf': 27, 'early_stopping_rounds': 28}. Best is trial 43 with value: 191104.37863729885.
Trial 48 finished with value: 202144.32793141753 and parameters: {'iterations': 780, 'learning_rate': 0.24056800772739648, 'depth': 10, 'l2_leaf_reg': 2.9151208029052, 'subsample': 0.6003847334191734, 'colsample_bylevel': 0.7395838685537687, 'min_data_in_leaf': 27, 'early_stopping_rounds': 28}. Best is trial 

[I 2025-06-30 17:29:14,958] Trial 49 finished with value: 247581.05166034037 and parameters: {'iterations': 891, 'learning_rate': 0.1649062241974827, 'depth': 8, 'l2_leaf_reg': 0.22802987601361496, 'subsample': 0.696435625302284, 'colsample_bylevel': 0.9100222982935362, 'min_data_in_leaf': 24, 'early_stopping_rounds': 41}. Best is trial 43 with value: 191104.37863729885.


Trial 49 finished with value: 247581.05166034037 and parameters: {'iterations': 891, 'learning_rate': 0.1649062241974827, 'depth': 8, 'l2_leaf_reg': 0.22802987601361496, 'subsample': 0.696435625302284, 'colsample_bylevel': 0.9100222982935362, 'min_data_in_leaf': 24, 'early_stopping_rounds': 41}. Best is trial 43 with value: 191104.37863729885.
Trial 49 finished with value: 247581.05166034037 and parameters: {'iterations': 891, 'learning_rate': 0.1649062241974827, 'depth': 8, 'l2_leaf_reg': 0.22802987601361496, 'subsample': 0.696435625302284, 'colsample_bylevel': 0.9100222982935362, 'min_data_in_leaf': 24, 'early_stopping_rounds': 41}. Best is trial 43 with value: 191104.37863729885.
Trial 49 finished with value: 247581.05166034037 and parameters: {'iterations': 891, 'learning_rate': 0.1649062241974827, 'depth': 8, 'l2_leaf_reg': 0.22802987601361496, 'subsample': 0.696435625302284, 'colsample_bylevel': 0.9100222982935362, 'min_data_in_leaf': 24, 'early_stopping_rounds': 41}. Best is tri

In [ ]:
##  BEST PARAMS OF CATBOOST OPTUNA
Best trial:
  Value (RMSE on validation set): 191104.3786
  Best Hyperparameters:
    iterations: 821
    learning_rate: 0.2801695218255082
    depth: 10
    l2_leaf_reg: 0.6670438837278988
    subsample: 0.6378212262786854
    colsample_bylevel: 0.7203717250461328
    min_data_in_leaf: 29
    early_stopping_rounds: 37

In [ ]:
###CATBOOSTOPTUNA WITHOUT ONEHOT 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor, Pool
import optuna
import logging
import sys

# --- 0. Data Loading and Preprocessing ---
print("--- Starting Data Loading and Preprocessing ---")
df = pd.read_csv("data/nonull_smalldata.csv")

# Print columns and dtypes for verification
print("\nOriginal DataFrame Columns:", df.columns.tolist())
print("\nOriginal DataFrame dtypes:\n", df.dtypes)

numeric_columns = ["bedroomCount", "toilet_and_bath", "habitableSurface", "facedeCount", "hasTerrace", "totalParkingCount"]
categorical_columns = ["type", "subtype", "province", "locality", "postCode", "buildingCondition", "epcScore"]

# Ensure all specified columns exist in the DataFrame
for col in numeric_columns + categorical_columns:
    if col not in df.columns:
        raise ValueError(f"Column '{col}' not found in DataFrame.")

# Define X and y BEFORE any column modifications for clear separation
X = df[numeric_columns + categorical_columns].copy() # Features
y = df["price"].copy() # Target

# Identify categorical feature indices for CatBoost
# We need the actual integer indices of these columns in the `X` DataFrame
cat_features_indices = [X.columns.get_loc(col) for col in categorical_columns]

print(f"\nIdentified categorical feature indices: {cat_features_indices}")
print(f"Corresponding categorical column names: {[X.columns[i] for i in cat_features_indices]}")

print("\nFeatures (X) shape:", X.shape)
print("Target (y) shape:", y.shape)

# Split data into training, validation, and test sets
# Ensure random_state is used for reproducibility
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=1234)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1234)

print("\nData splitting complete.")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# Create CatBoost Pool objects, passing the categorical feature indices
train_pool = Pool(X_train, y_train, cat_features=cat_features_indices)
val_pool = Pool(X_val, y_val, cat_features=cat_features_indices)
test_pool = Pool(X_test, y_test, cat_features=cat_features_indices)

print("--- Data Preprocessing Complete ---")

# --- 2. Define the Optuna Objective Function ---
def objective(trial):
    """
    Objective function for Optuna to optimize CatBoost hyperparameters.
    It returns the validation RMSE for a given set of hyperparameters.
    """
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 0.3),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-2, 10),
        "subsample": trial.suggest_uniform("subsample", 0.6, 1.0),
        "colsample_bylevel": trial.suggest_uniform("colsample_bylevel", 0.6, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 30),
        "loss_function": "RMSE",
        "eval_metric": "RMSE",
        "random_seed": 42,
        "verbose": 0,
        "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 10, 50),
    }

    model = CatBoostRegressor(**params)

    pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")

    try:
        model.fit(
            train_pool,
            eval_set=val_pool,
            early_stopping_rounds=params["early_stopping_rounds"],
            callbacks=[pruning_callback],
            verbose=0 # Suppress verbose output during tuning
        )
    except optuna.exceptions.TrialPruned:
        raise

    val_predictions = model.predict(X_val) # Predict on X_val directly for evaluation
    rmse = np.sqrt(mean_squared_error(y_val, val_predictions))

    return rmse

# --- 3. Set up and Run Optuna Study ---
if __name__ == "__main__":
    optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(seed=42),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
    )

    print("\n--- Starting Optuna optimization ---")
    study.optimize(objective, n_trials=50, timeout=600) # Run 50 trials or for 10 minutes

    print("\n--- Optimization finished ---")
    print(f"Number of finished trials: {len(study.trials)}")
    print(f"Number of pruned trials: {len(study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.PRUNED]))}")

    print("\nBest trial:")
    trial = study.best_trial

    print(f"  Value (RMSE on validation set): {trial.value:.4f}")
    print("  Best Hyperparameters:")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    # --- 4. Train the final model with the best hyperparameters ---
    print("\n--- Training final model with best hyperparameters ---")
    best_params = trial.params
    best_params_for_final_model = {k: v for k, v in best_params.items() if k != "early_stopping_rounds"}

    best_params_for_final_model["iterations"] = 10000 # Set a high number, rely on early stopping
    best_params_for_final_model["verbose"] = 100 # Show some progress for the final training

    final_cat_regressor = CatBoostRegressor(**best_params_for_final_model)

    final_cat_regressor.fit(
        train_pool,
        eval_set=val_pool,
        early_stopping_rounds=best_params["early_stopping_rounds"], # Use the best found early stopping
        verbose=best_params_for_final_model["verbose"],
    )

    print(f"Best iteration of final model: {final_cat_regressor.get_best_iteration()}")

    # --- 5. Evaluate the final model on the unseen test set ---
    final_predictions = final_cat_regressor.predict(test_pool)

    final_mse = mean_squared_error(y_test, final_predictions)
    final_rmse = np.sqrt(final_mse)
    final_r2 = r2_score(y_test, final_predictions)

    print(f"\n--- Final Model Performance on Test Set ---")
    print(f"  MSE: {final_mse:.4f}")
    print(f"  RMSE: {final_rmse:.4f}")
    print(f"  R-squared: {final_r2:.4f}")

    # Optionally, visualize the optimization process
    try:
        from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice
        print("\n--- Generating Optuna Visualizations (requires plotly and kaleido) ---")
        fig_history = plot_optimization_history(study)
        fig_history.show()

        fig_parallel = plot_parallel_coordinate(study)
        fig_parallel.show()

        fig_slice = plot_slice(study)
        fig_slice.show()
    except ImportError:
        print("\nInstall plotly and kaleido for Optuna visualizations: pip install plotly kaleido")

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import logging
import sys

# Import the specific callbacks from their correct locations for XGBoost 3.0.2
from optuna.integration.xgboost import XGBoostPruningCallback
from xgboost.callback import EarlyStopping # Directly from xgboost.callback


# --- 0. Data Loading and Preprocessing ---
print("--- Starting Data Loading and Preprocessing ---")
df = pd.read_csv("data/nonull_smalldata.csv")

# Print columns and dtypes for verification
print("\nOriginal DataFrame Columns:", df.columns.tolist())
print("\nOriginal DataFrame dtypes:\n", df.dtypes)

numeric_columns = ["bedroomCount", "toilet_and_bath", "habitableSurface", "facedeCount", "hasTerrace", "totalParkingCount"]
categorical_columns = ["type", "subtype", "province", "locality", "postCode", "buildingCondition", "epcScore"]

# Ensure all specified categorical columns actually exist in the DataFrame
for col in categorical_columns:
    if col not in df.columns:
        raise ValueError(f"Categorical column '{col}' not found in DataFrame.")

# Store 'price' column before potentially modifying df
price_column_data = df['price']

# One-Hot Encode categorical columns
encoder = OneHotEncoder(sparse_output=False, drop="first")
one_hot_encoded = encoder.fit_transform(df[categorical_columns])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

# Concatenate one-hot encoded features with numeric features
df_numeric = df[numeric_columns]
X = pd.concat([df_numeric.reset_index(drop=True), one_hot_df.reset_index(drop=True)], axis=1)

y = price_column_data.reset_index(drop=True)

print("\nFeatures (X) shape after encoding:", X.shape)
print("Target (y) shape:", y.shape)

# Split data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=1234)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1234)

print("\nData splitting complete.")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

print("--- Data Preprocessing Complete ---")

# --- 2. Define the Optuna Objective Function for XGBoost (Correct for XGBoost 3.0.2) ---
def objective(trial):
    """
    Objective function for Optuna to optimize XGBoost hyperparameters.
    It returns the validation RMSE for a given set of hyperparameters.
    """
    # 2.1. Suggest hyperparameters for XGBoost
    params = {
        "objective": "reg:squarederror",  # For regression tasks (MSE)
        "eval_metric": "rmse",            # Metric for early stopping
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000), # Max number of boosting rounds
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_uniform("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_loguniform("gamma", 1e-8, 1.0),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 1e-2, 10),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 1.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 1.0),
        "random_state": 42,
        "n_jobs": -1, # Use all available cores
    }

    # 2.2. Initialize and Train XGBoost Regressor with suggested parameters
    model = XGBRegressor(**params)

    # Optuna's XGBoostPruningCallback for pruning (requires 'callbacks' argument in fit)
    pruning_callback = XGBoostPruningCallback(trial, "validation_0-rmse")

    # XGBoost's native EarlyStopping callback (requires 'callbacks' argument in fit)
    early_stopping_rounds_for_trial = 50 # A fixed value for early stopping in Optuna trials
    early_stopping_callback = EarlyStopping(
        rounds=early_stopping_rounds_for_trial,
        metric_name="validation_0-rmse",
        data_name="validation_0",
        maximize=False # Minimize RMSE
        # 'verbose' argument is not used here for newer XGBoost versions
    )

    try:
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)], # Pass eval_set as a list of tuples
           # callbacks=[pruning_callback, early_stopping_callback], # Pass all callbacks as a list
            verbose=False # Suppress training output for cleaner Optuna logs during tuning
        )
    except optuna.exceptions.TrialPruned:
        raise

    # 2.3. Make predictions on the validation set
    # XGBoost's `predict` method automatically uses the best iteration if early stopping occurred.
    val_predictions = model.predict(X_val)

    # 2.4. Calculate the evaluation metric (RMSE)
    rmse = np.sqrt(mean_squared_error(y_val, val_predictions))

    return rmse

# --- 3. Set up and Run Optuna Study ---
if __name__ == "__main__":
    # Set up Optuna logging (optional, but good for debugging)
    optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

    study = optuna.create_study(
        direction="minimize",  # We want to minimize RMSE
        sampler=optuna.samplers.TPESampler(seed=42), # Tree-structured Parzen Estimator (TPE) sampler
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=10) # Pruner for early stopping unpromising trials
    )

    print(f"Starting Optuna optimization for XGBoost (Version {xgb.__version__})...")
    # Optimize the objective function
    study.optimize(objective, n_trials=50, timeout=600) # Run 50 trials or for 600 seconds (10 minutes)

    print("\nOptimization finished.")
    print(f"Number of finished trials: {len(study.trials)}")
    print(f"Number of pruned trials: {len(study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.PRUNED]))}")

    print("\nBest trial:")
    trial = study.best_trial

    print(f"  Value (RMSE on validation set): {trial.value:.4f}")
    print("  Best Hyperparameters:")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    # --- 4. Train the final model with the best hyperparameters ---
    print("\nTraining final model with best hyperparameters...")
    best_params = trial.params
    # Create a copy and ensure n_estimators is high, letting early stopping handle the actual rounds
    final_model_params = {k: v for k, v in best_params.items()}
    final_model_params["n_estimators"] = 10000 # Set a high number, rely on early stopping
    final_model_params["verbose"] = False # Suppress verbose output for final training, or set to an integer

    final_xgb_regressor = XGBRegressor(**final_model_params)

    # Define early stopping callback for the final model training
    final_early_stopping_rounds = 50 # You can keep this fixed or use a value from tuning
    final_early_stopping_callback = EarlyStopping(
        rounds=final_early_stopping_rounds,
        metric_name="validation_0-rmse",
        data_name="validation_0",
        maximize=False,
        verbose=True # Set to True to see final model training progress
    )

    final_xgb_regressor.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)], # Use validation set for early stopping
        #callbacks=[final_early_stopping_callback], # Pass the callback here
       # verbose=False # Verbosity of fitting is controlled by the callback's verbose setting
    )

    # The best iteration from early stopping is automatically used for predictions
    print(f"Best iteration of final model: {final_xgb_regressor.best_iteration}")

    # --- 5. Evaluate the final model on the unseen test set ---
    final_predictions = final_xgb_regressor.predict(X_test)

    final_mse = mean_squared_error(y_test, final_predictions)
    final_rmse = np.sqrt(final_mse)
    final_r2 = r2_score(y_test, final_predictions)

    print(f"\n--- Final Model Performance on Test Set ---")
    print(f"  MSE: {final_mse:.4f}")
    print(f"  RMSE: {final_rmse:.4f}")
    print(f"  R-squared: {final_r2:.4f}")

    # Optionally, visualize the optimization process
    try:
        from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice
        print("\n--- Generating Optuna Visualizations (requires plotly and kaleido) ---")
        fig_history = plot_optimization_history(study)
        fig_history.show()

        fig_parallel = plot_parallel_coordinate(study)
        fig_parallel.show()

        fig_slice = plot_slice(study)
        fig_slice.show()
    except ImportError:
        print("\nInstall plotly and kaleido for Optuna visualizations: pip install plotly kaleido")

--- Starting Data Loading and Preprocessing ---

Original DataFrame Columns: ['type', 'subtype', 'bedroomCount', 'toilet_and_bath', 'province', 'locality', 'postCode', 'habitableSurface', 'buildingCondition', 'facedeCount', 'hasTerrace', 'epcScore', 'price', 'totalParkingCount']

Original DataFrame dtypes:
 type                  object
subtype               object
bedroomCount         float64
toilet_and_bath      float64
province              object
locality              object
postCode               int64
habitableSurface     float64
buildingCondition     object
facedeCount          float64
hasTerrace              bool
epcScore              object
price                float64
totalParkingCount    float64
dtype: object

Features (X) shape after encoding: (56005, 4620)
Target (y) shape: (56005,)


[I 2025-06-30 18:36:20,821] A new study created in memory with name: no-name-2fe33898-63d0-45f2-8536-0ec8089acdfe



Data splitting complete.
X_train shape: (39203, 4620), y_train shape: (39203,)
X_val shape: (8401, 4620), y_val shape: (8401,)
X_test shape: (8401, 4620), y_test shape: (8401,)
--- Data Preprocessing Complete ---
A new study created in memory with name: no-name-2fe33898-63d0-45f2-8536-0ec8089acdfe
A new study created in memory with name: no-name-2fe33898-63d0-45f2-8536-0ec8089acdfe
A new study created in memory with name: no-name-2fe33898-63d0-45f2-8536-0ec8089acdfe
A new study created in memory with name: no-name-2fe33898-63d0-45f2-8536-0ec8089acdfe
Starting Optuna optimization for XGBoost (Version 3.0.2)...


C:\Users\preet\AppData\Local\Temp\ipykernel_29912\3214107848.py:71: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 0.3),
C:\Users\preet\AppData\Local\Temp\ipykernel_29912\3214107848.py:73: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "subsample": trial.suggest_uniform("subsample", 0.6, 1.0),
C:\Users\preet\AppData\Local\Temp\ipykernel_29912\3214107848.py:74: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.6, 1.0),
C:\Users

Trial 0 finished with value: 259665.94784065822 and parameters: {'n_estimators': 437, 'learning_rate': 0.22648248189516848, 'max_depth': 8, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'gamma': 1.7699302940633311e-07, 'min_child_weight': 0.014936568554617643, 'reg_alpha': 0.08499808989182997, 'reg_lambda': 0.0006440507553993703}. Best is trial 0 with value: 259665.94784065822.
Trial 0 finished with value: 259665.94784065822 and parameters: {'n_estimators': 437, 'learning_rate': 0.22648248189516848, 'max_depth': 8, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'gamma': 1.7699302940633311e-07, 'min_child_weight': 0.014936568554617643, 'reg_alpha': 0.08499808989182997, 'reg_lambda': 0.0006440507553993703}. Best is trial 0 with value: 259665.94784065822.
Trial 0 finished with value: 259665.94784065822 and parameters: {'n_estimators': 437, 'learning_rate': 0.22648248189516848, 'max_depth': 8, 'subsample': 0.8394633936788146, 'colsample_

C:\Users\preet\AppData\Local\Temp\ipykernel_29912\3214107848.py:71: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 0.3),
C:\Users\preet\AppData\Local\Temp\ipykernel_29912\3214107848.py:73: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "subsample": trial.suggest_uniform("subsample", 0.6, 1.0),
C:\Users\preet\AppData\Local\Temp\ipykernel_29912\3214107848.py:74: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.6, 1.0),
C:\Users

Trial 1 finished with value: 346406.8839108318 and parameters: {'n_estimators': 737, 'learning_rate': 0.001124579825911934, 'max_depth': 10, 'subsample': 0.9329770563201687, 'colsample_bytree': 0.6849356442713105, 'gamma': 2.8483918709107956e-07, 'min_child_weight': 0.03549878832196503, 'reg_alpha': 2.716051144654844e-06, 'reg_lambda': 0.00015777981883364995}. Best is trial 0 with value: 259665.94784065822.
Trial 1 finished with value: 346406.8839108318 and parameters: {'n_estimators': 737, 'learning_rate': 0.001124579825911934, 'max_depth': 10, 'subsample': 0.9329770563201687, 'colsample_bytree': 0.6849356442713105, 'gamma': 2.8483918709107956e-07, 'min_child_weight': 0.03549878832196503, 'reg_alpha': 2.716051144654844e-06, 'reg_lambda': 0.00015777981883364995}. Best is trial 0 with value: 259665.94784065822.
Trial 1 finished with value: 346406.8839108318 and parameters: {'n_estimators': 737, 'learning_rate': 0.001124579825911934, 'max_depth': 10, 'subsample': 0.9329770563201687, 'col

TypeError: EarlyStopping.__init__() got an unexpected keyword argument 'verbose'

In [3]:
import xgboost as xgb
print(f"Your XGBoost version: {xgb.__version__}")

Your XGBoost version: 3.0.2


In [ ]:
# BEST PARAMS OF XGBOOSTOPTUNA
Best trial:
  Value (RMSE on validation set): 259665.9478
  Best Hyperparameters:
    n_estimators: 437
    learning_rate: 0.22648248189516848
    max_depth: 8
    subsample: 0.8394633936788146
    colsample_bytree: 0.6624074561769746
    gamma: 1.7699302940633311e-07
    min_child_weight: 0.014936568554617643
    reg_alpha: 0.08499808989182997
    reg_lambda: 0.0006440507553993703

In [ ]:
xgb_regressor = XGBRegressor(
    objective='reg:squarederror', # For regression tasks, specify the objective
    n_estimators=437,             # Number of boosting rounds
    learning_rate=0.22648248189516848,            # Step size shrinkage
    max_depth=8,                  # Maximum depth of a tree
    subsample=0.8394633936788146,                # Subsample ratio of the training instance
    colsample_bytree=0.6624074561769746,         # Subsample ratio of columns
    random_state=42,              # For reproducibility
    n_jobs=-1 ,
)
'''gamma=1.7699302940633311e-07,
min_child_weight= 0.014936568554617643,
reg_alpha= 0.08499808989182997,
reg_lambda= 0.0006440507553993703,'''
# Use all available CPU cores


# Fit the model to the training data
xgb_regressor.fit(X_train, y_train)

# Make predictions on the test data
xgb_predictions = xgb_regressor.predict(X_test)

# Evaluate the XGBoost model
xgb_mse = mean_squared_error(y_test, xgb_predictions)
xgb_r2 = r2_score(y_test, xgb_predictions)

print(f"XGBoost MSE: {xgb_mse}")
predictions = xgb_regressor.predict(X_test)
accu = r2_score(y_test, predictions)
print("Accuracy of test:", accu)

pred = xgb_regressor.predict(X_train)
accu = r2_score(y_train, pred)
print("Accuracy of train:", accu)

NameError: name 'X_train' is not defined